In [1]:
import cv2
import os
import tensorflow as tf
import numpy as np
import opendatasets as od

from tensorflow import keras

In [2]:
print(os.getcwd())
if 'Emotions' not in os.getcwd():
    os.chdir(os.getcwd() + '/Emotions-ImageClassification-TensorFlow')

/teamspace/studios/this_studio


In [3]:
os.getcwd()

'/teamspace/studios/this_studio/Emotions-ImageClassification-TensorFlow'

In [4]:
train = tf.data.Dataset.load('data_compressed/train')
test = tf.data.Dataset.load('data_compressed/test')
val = tf.data.Dataset.load('data_compressed/val')

In [5]:
[print(len(i)) for i in [train, test, val]]

28272
3534
3534


[None, None, None]

In [6]:
print(train)

<_LoadDataset element_spec=(TensorSpec(shape=(32, 32, 1), dtype=tf.float32, name=None), TensorSpec(shape=(6,), dtype=tf.float32, name=None))>


In [10]:
train = train.batch(batch_size=32).cache().prefetch(tf.data.AUTOTUNE)
test = test.batch(batch_size=32).cache().prefetch(tf.data.AUTOTUNE)
val = val.batch(batch_size=32).cache().prefetch(tf.data.AUTOTUNE)

In [11]:
print(train)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 32, 32, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 6), dtype=tf.float32, name=None))>


In [12]:
model = keras.Sequential([
    keras.layers.Input(shape=(32, 32, 1)),
    keras.layers.Conv2D(32, kernel_size=3, activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(6, activation='softmax')
])

model.compile(optimizer='adam', 
              loss=keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])


model.fit(train, epochs=1, validation_data=val)


2024-05-08 07:23:23.617004: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: DATA_LOSS: corrupted record at 0 (Is this even a TFRecord file?)
	 [[{{node IteratorGetNext}}]]


DataLossError: Graph execution error:

Detected at node IteratorGetNext defined at (most recent call last):
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 737, in start

  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/asyncio/base